In [1]:
import datetime as dt
import time
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM
from keras import optimizers
from keras import backend as K

Using TensorFlow backend.


In [2]:
# plotting options
%matplotlib inline
sns.set_style('whitegrid')
colors = sns.color_palette('deep', 8)
sns.set_palette(colors)
figsize1 = (17,6)
figsize2 = (17,4)
figsize3 = (17,3)
dpi=200
title_fontsize=16
ax_fontsize=12

In [3]:
def shift_data(df, n_shifts):
    # copy df, shift cols in a new df and concat them
    df_shifted = df.copy()
    for i in range(n_shifts):
        df = pd.concat([df, df_shifted.shift(i+1)], axis=1)
        df.dropna(axis=0, inplace=True)
    
    # rename columns    
    cols = list(df.columns)[1:]
    for i, col in enumerate(cols):
        cols[i] = 'shift_{}'.format(i+1)
    cols = [df.columns[0]] + cols
    df.columns = cols
    return df

def shape_data(data):
    data = np.array(data).reshape(data.shape[0], data.shape[1], 1)
    return data

In [4]:
def build_lstm(X, units, activation, optimizer, lr, use_bias):
    model = Sequential()
    model.add(LSTM(units=units,
                   activation=activation,
                   use_bias=use_bias,
                   input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1))

    model.compile(
        loss='mean_squared_error',
        metrics=['mse'],
        optimizer=optimizer
    )
    return model

def lstm_forecast(model, X, y, window, epochs,
                  verbose=0, callbacks=None, monitor=False):
    start = 0
    stop = window
    preds = []
    observed = ts.eth_close.values[-(len(X)-window):]
    
    for i in range(len(y) - window):
                
        # set train data to window sized chunk
        X_train, y_train = X[start:stop], y[start:stop]
        # set test data to one step after train data
        X_test, y_test = X[stop:stop + 1], y[stop:stop + 1]
        
        # fit model
        start_time = time.time()
        model.fit(
            X_train,
            y_train,
            epochs=epochs,
            verbose=verbose,
            callbacks=callbacks,
            validation_split=0.1
        )
        # make & store prediction, step foreward
        yhat = model.predict(X_test)
        yhat = scaler.inverse_transform(np.array(yhat[0][0]).reshape(-1,1))
        preds.append(yhat)
        
        # monitor function without model verbosity
        if monitor is True:
            if start != 0 and start % 100  == 0:
                print("iteration {} predicted: {} (observed: {}) in {:.2f} seconds".format(
                    start, yhat, observed[start], (time.time()-start_time)))
            
        start += 1
        stop += 1
    return preds

def lstm_rmse(preds):
    rmse = np.sqrt(mean_squared_error(ts.eth_close.values[-len(preds):],
                                      np.array(preds).reshape(len(preds),)))
    return rmse

In [5]:
raw = pd.read_json('df.json', orient='split')
ts = pd.DataFrame(raw.eth_close.copy())
ts.head()

,eth_close
2015-08-06,3.00
2015-08-07,1.20
2015-08-08,1.20
2015-08-09,1.20
2015-08-10,0.99


In [6]:
scaler = MinMaxScaler(feature_range=(0,1))
ts_scaled = scaler.fit_transform(ts.values)
ts = pd.DataFrame(ts_scaled).set_index(ts.index.values)
ts.columns = ['unshifted']

ts = shift_data(ts, n_shifts=1)
X = pd.DataFrame(ts.shift_1)
X = shape_data(X)
y = ts.unshifted

print(X.shape, y.shape)

(1173, 1, 1) (1173,)


In [7]:
def lstm_expanding_window(model, X, y, epochs=500, verbose=0):
    
    preds = []
    observed = ts.unshifted.values[-len(X):]
    
    early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=25,
    verbose=0)
    
    for i in range(len(y)):
        X_train, y_train = X[:i], y[:i]
        X_test, y_test = X[i:i+1], y[i:i+1]
        
        start_time = time.time()
        model.fit(
            X_train,
            y_train,
            epochs=epochs,
            verbose=verbose,
            callbacks=[early_stopping],
            validation_split=0.1
        )
        # make & store prediction, step foreward
        yhat = model.predict(X_test)
        yhat = scaler.inverse_transform(np.array(yhat[0][0]).reshape(-1,1))
        preds.append(yhat)
    
    return preds

def lstm_rmse(preds):
    rmse = np.sqrt(mean_squared_error(ts.unshifted.values[-len(preds):],
                                      np.array(preds).reshape(len(preds),)))
    return rmse

In [8]:
lstm = build_lstm(
    X=X,
    optimizer='RMSprop',
    lr=0.0001,
    activation='tanh',
    units=32,
    use_bias=True
)